In [ ]:
# Comandos de instalación (se ejecutan en la celda de Colab)
!pip3 install pycrypto
!pip3 uninstall PyCrypto  # Parece que desinstala la versión vieja
!pip3 install -U PyCryptodome # E instala la versión moderna
!apt-get install gcc python-dev libgmp3-dev # Dependencias de sistema
!pip3 install ECPy
!pip3 install ed25519

2. Generación de Números Primos y Hashing

In [ ]:
from Crypto.Util import number
# Generar un número primo grande de 1024 bits
print(number.getPrime(N=1024, randfunc=None))

from Crypto.Hash import SHA256
# Hash SHA256 de un texto
h = SHA256.new()
h.update(b'Prueba11')
print(h.hexdigest())

3. Cifrado Simétrico (DES, AES, ARC4)

DES (Modo ECB - Inseguro):

In [ ]:
from Crypto.Cipher import DES
des = DES.new('01234567'.encode("utf8"), DES.MODE_ECB)
text = 'abcdefwh'
cipher_text = des.encrypt(text.encode("utf8"))
print(cipher_text)
des.decrypt(cipher_text)

DES (Modo CFB con Vector de Inicialización):

In [ ]:
from Crypto.Cipher import DES
from Crypto import Random
iv = Random.get_random_bytes(8)
des1 = DES.new('01234567'.encode("utf8"), DES.MODE_CFB, iv)
des2 = DES.new('01234567'.encode("utf8"), DES.MODE_CFB, iv)
text = 'abcdefghijklmnop'
cipher_text = des1.encrypt(text.encode("utf8"))
print(cipher_text)
des2.decrypt(cipher_text)

AES (Modo CBC):

In [ ]:
from Crypto.Cipher import AES
# Clave de 16 bytes y IV de 16 bytes
obj = AES.new('This is a keyabc'.encode("utf8"), AES.MODE_CBC, 'This is an IV456'.encode("utf8"))
message = "Text Plain Test1"
ciphertext = obj.encrypt(message.encode("utf8"))
print(ciphertext)
obj2 = AES.new('This is a keyabc'.encode("utf8"), AES.MODE_CBC, 'This is an IV456'.encode("utf8"))
obj2.decrypt(ciphertext)

ARC4 (Cifrado de flujo):

In [ ]:
from Crypto.Cipher import ARC4
obj1 = ARC4.new('01234567'.encode("utf8"))
obj2 = ARC4.new('01234567'.encode("utf8"))
text = 'abcdefghijklmnop'
cipher_text = obj1.encrypt(text.encode("utf8"))
print(cipher_text)
obj2.decrypt(cipher_text)

4. Cifrado Asimétrico (RSA)

In [ ]:
Generar claves y guardar en archivos:

In [ ]:
from Crypto.PublicKey import RSA

key = RSA.generate(2048)
private_key = key.export_key()
with open("private.pem", "wb") as file_out:
    file_out.write(private_key)

public_key = key.publickey().export_key()
with open("receiver.pem", "wb") as file_out:
    file_out.write(public_key)

Cifrado Híbrido (AES + RSA): Cifra una clave de sesión AES usando RSA, y los datos con AES.

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

data = "Conozco alinenígenas".encode("utf-8")
file_out = open("encrypted_data.bin", "wb")

recipient_key = RSA.import_key(open("receiver.pem").read())
session_key = get_random_bytes(16)

# Cifrar la clave de sesión con RSA
cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

# Cifrar los datos con AES
cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(data)
[ file_out.write(x) for x in (enc_session_key, cipher_aes.nonce, tag, ciphertext) ]
file_out.close()

Firmas Digitales RSA:

In [ ]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from hashlib import sha512

msg = b'A message for signing'
keyPair = RSA.generate(bits=1024)
hash = int.from_bytes(sha512(msg).digest(), byteorder='big')
signature = pow(hash, keyPair.d, keyPair.n)
print("Signature:", hex(signature))

# Verificación
hashFromSignature = pow(signature, keyPair.e, keyPair.n)
print("Signature valid:", hash == hashFromSignature)

5. Curvas Elípticas (ECC) y Ed25519

ECDSA con secp256k1 (Curva de Bitcoin):

In [ ]:
from ecpy.curves import Curve, Point
from ecpy.keys import ECPublicKey, ECPrivateKey
from ecpy.ecdsa import ECDSA

cv = Curve.get_curve('secp256k1')
# (El código define puntos específicos P y claves privadas para firmar)
signer = ECDSA()
# ... firma y verificación ...

Ed25519 (Firma rápida y segura):

In [ ]:
import ed25519
privKey, pubKey = ed25519.create_keypair()
msg = b'Message for Ed25519 signing'
signature = privKey.sign(msg, encoding='hex')
try:
    pubKey.verify(signature, msg, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")